In [112]:
import pandas as pd
import polars as pl

In [87]:
# 市区町村抽出の正規表現
# from https://qiita.com/zakuroishikuro/items/066421bce820e3c73ce9
EXTRACT_REG=r"(...??[都道府県])((?:旭川|伊達|石狩|盛岡|奥州|田村|南相馬|那須塩原|東村山|武蔵村山|羽村|十日町|上越|富山|野々市|大町|蒲郡|四日市|姫路|大和郡山|廿日市|下松|岩国|田川|大村)市|.+?郡(?:玉村|大町|.+?)[町村]|.+?市.+?区|.+?[市区町村])(.*)"

### 人口

In [307]:
# 独特なExcelなので、頑張って読み込む。
jinko_df = (
  pl.from_dataframe(
    pd.read_excel(
      "data/2303ssjin.xlsx",
      usecols="A:G", skiprows=5,
      names=[
        "団体コード", "都道府県名", "市区町村名", "人口(男)", "人口(女)", "人口(計)", "世帯数"
      ]
    )
  )
  .with_columns([
    # 正規表現から市町村列を追加する (地価データを結合する用)
    (pl.col("都道府県名") + pl.col("市区町村名"))
      .str.extract(EXTRACT_REG, 2)
      .alias("市区町村"),
  ])
	.filter(
    # 人口が0の場所を除く（北方領土とか）
    pl.col("人口(計)") > 0
  )
  .filter(
    # 市町村がうまく拾えないレコード（例: 県全体・◯◯郡 など）を除く
    pl.col("市区町村").is_not_null()
  )
  .filter([
    # 政令指定都市は区単位と市単位両方あって二重にカウントしてしまうので、力技で抜く
    ~pl.col("市区町村").str.contains(
      "(静岡市|神戸市|広島市|千葉市|熊本市|名古屋市|新潟市|横浜市|京都市|岡山市|大阪市|堺市|福岡市|北九州市|川崎市|相模原市|浜松市|さいたま市|札幌市|仙台市)$",
    )
  ])
  .select([
    "都道府県名", "市区町村", "人口(計)"
  ])
)

jinko_df.to_pandas()

,都道府県名,市区町村,人口(計)
0,北海道,札幌市中央区,244032.0
1,北海道,札幌市北区,285671.0
2,北海道,札幌市東区,261288.0
3,北海道,札幌市白石区,213006.0
4,北海道,札幌市豊平区,225221.0
...,...,...,...
1892,沖縄県,島尻郡久米島町,7413.0
1893,沖縄県,島尻郡八重瀬町,32630.0
1894,沖縄県,宮古郡多良間村,1085.0
1895,沖縄県,八重山郡竹富町,4288.0


In [311]:
jinko_df.write_csv("export/jinko.csv")

### 地価

In [306]:
# 変わったcsvなので、頑張って加工する
chika_df = (
  pl.from_dataframe(
    pd.read_csv("data/chika.csv",encoding="cp932")
  )
  .rename({
    "住所（住居表示・所在地番）":"住所",
    "〒R4←R3←R2":"地価推移"
  })
  .select([
    "住所", "地価推移"
  ])
  .with_columns([
    pl.col("住所")
      .str.extract(EXTRACT_REG, 1)
      .alias("都道府県名"),
    pl.col("住所")
      .str.replace(" ", "")
      .str.extract(EXTRACT_REG, 2)
      .alias("市区町村"),
    pl.col("地価推移")
      .str.extract(r"R4=(\d*\.?\d+\.?)", 1)
      .cast(pl.Float32)
      .alias("最新地価"),
  ])
  .select([
    "都道府県名", "市区町村", "住所", "最新地価"
  ])
)
chika_df.to_pandas()

,都道府県名,市区町村,住所,最新地価
0,北海道,札幌市中央区,北海道 札幌市中央区大通西28-2-5,397.000000
1,北海道,札幌市中央区,北海道 札幌市中央区南13条西13-2-10,170.000000
2,北海道,札幌市中央区,北海道 札幌市中央区南10条西9-1-48,160.000000
3,北海道,札幌市中央区,北海道 札幌市中央区南18条西8-1-43,186.000000
4,北海道,札幌市中央区,北海道 札幌市中央区双子山2-3-12,52.299999
...,...,...,...,...
25988,沖縄県,島尻郡南風原町,沖縄県 島尻郡南風原町(19街区5画地外),166.000000
25989,沖縄県,島尻郡八重瀬町,沖縄県 島尻郡八重瀬町字東風平西徳枡原1404番5,70.800003
25990,沖縄県,島尻郡八重瀬町,沖縄県 島尻郡八重瀬町字東風平東風平原169番7,58.799999
25991,沖縄県,島尻郡八重瀬町,沖縄県 島尻郡八重瀬町字宜次宜次原37番,45.500000


In [312]:
chika_df.write_csv("export/chika.csv")

### 結合

In [333]:
# 各市町村ごとに、地価の中央値を結合する
df_city = (jinko_df
  .join(chika_df, on=["都道府県名", "市区町村"], how="left")
  .group_by(["都道府県名", "市区町村"])
  .agg([
    pl.mean("人口(計)").alias("人口"),
    pl.median("最新地価").alias("地価中央値"),
  ])
)
df_city

都道府県名,市区町村,人口,地価中央値
str,str,f64,f32
"""北海道""","""滝川市""",38062.0,11.3
"""北海道""","""歌志内市""",2790.0,2.35
"""北海道""","""松前郡松前町""",6231.0,null
"""北海道""","""夕張郡由仁町""",4783.0,null
"""北海道""","""上川郡東川町""",8601.0,null
"""北海道""","""上川郡下川町""",3027.0,2.95
"""北海道""","""宗谷郡猿払村""",2637.0,null
"""北海道""","""紋別郡滝上町""",2363.0,1.9
"""北海道""","""網走郡大空町""",6771.0,6.9


In [340]:
# 各都道府県ごとに、地価を平均して結合する
df_pref = (jinko_df
	.join(chika_df, on=["都道府県名", "市区町村"], how="left")
	.group_by(["都道府県名"])
	.agg([
		pl.sum("人口(計)").alias("人口"),
		pl.median("最新地価").alias("地価中央値")
		# 中央値だと地価が若干高く見積もられるのが気になるなら、percentileを使う
    # pl.col("最新地価").quantile(0.1).alias("地価中央値")
	])
)


In [341]:
df_city_fill = (df_city
  .join(df_pref, on="都道府県名", how="left", suffix="_pref")
  .with_columns([
    # pl.col(["都道府県名", "市区町村", "人口"]),
    pl.col("地価中央値").is_not_null().alias("地価データ有無"),
  ])
  .with_columns([
    pl.when(pl.col("地価データ有無"))
      .then(pl.col("地価中央値"))
      .otherwise(pl.col("地価中央値_pref"))
      .alias("地価中央値")
  ])
)

In [342]:
df_city_pr = (df_city_fill
  .with_columns([
    (pl.col("人口") * pl.col("地価中央値") / 1_000_000).alias("人口地価積"),
  ])
  .sort("人口地価積", descending=True)
)

In [343]:
(df_city_pr
 .select(["都道府県名", "市区町村", "人口", "地価中央値", "人口地価積"])
 .write_csv("export/output.csv")
)

In [348]:
df_pref_rank = (df_city_pr
  .select([
    "都道府県名", "市区町村", "人口地価積", "人口", "地価中央値", "地価データ有無"
  ])
  .group_by(["都道府県名"])
  .agg([
    (pl.sum("人口") / 10_000).alias("人口(万)"),
    pl.col("人口地価積").quantile(0.99).alias("人口地価積_99%"),
    pl.col("人口地価積").quantile(0.90).alias("人口地価積_90%"),
    pl.col("人口地価積").quantile(0.75).alias("人口地価積_75%"),
    pl.col("人口地価積").quantile(0.50).alias("人口地価積_50%"),
    pl.col("人口地価積").quantile(0.25).alias("人口地価積_25%"),
    pl.col("人口地価積").quantile(0.10).alias("人口地価積_10%"),
  ])
  # 90%値を使う。なぜならば、殆どの人は割と栄えている場所を軸に話をしているため。
  .sort("人口地価積_90%", descending=True)
)
df_pref_rank.to_pandas()

,都道府県名,人口(万),人口地価積_99%,人口地価積_90%,人口地価積_75%,人口地価積_50%,人口地価積_25%,人口地価積_10%
0,東京都,1384.1665,688.047450,376.235680,225.842461,49.802186,8.270336,0.127896
1,神奈川県,921.2003,115.395150,75.554340,53.892560,34.762620,4.762958,0.962871
2,大阪府,830.4284,163.281215,48.342744,32.880749,16.498621,6.599183,1.442387
3,兵庫県,545.9867,138.562452,41.362464,20.668083,2.176576,1.066100,0.482976
4,京都府,250.1269,93.957100,39.501252,22.039424,5.557520,0.933203,0.271189
5,埼玉県,738.1035,123.966575,37.312058,20.401422,7.303704,1.672481,0.427456
6,愛知県,742.6432,90.797520,35.930130,26.769741,8.454235,3.540383,1.425645
7,広島県,258.0270,65.761670,29.511082,11.738420,3.806876,0.721552,0.436775
8,千葉県,631.0075,118.961634,27.717333,11.659280,2.328523,0.587239,0.182437
9,静岡県,363.3773,39.401472,17.977114,7.422466,3.780880,1.174419,0.423459


In [349]:
def rank_func(val: float) -> str:
  if val > 2.0:
    return "#1"
  if val > 1.50:
    return "#2"
  if val > 1.00:
    return "#3"
  if val > 0.75:
    return "#4"
  if val > 0.50:
    return "#5"
  if val > 0.25:
    return "#6"
  return "#7"

df_tier = (df_pref_rank
  .with_columns([
    pl.col("人口地価積_90%").log10().alias("log10"),
    pl.col("人口地価積_90%").log10().map_elements(rank_func).alias("Tier")
  ])
  .group_by(["Tier"])
  .agg([
    pl.col("都道府県名")
  ])
  .select([
    "Tier",
    pl.col("都道府県名").list.join(separator=", ")
  ])
  .sort("Tier")
  .to_pandas()
)
df_tier

,Tier,都道府県名
0,#1,東京都
1,#2,"神奈川県, 大阪府, 兵庫県, 京都府, 埼玉県, 愛知県"
2,#3,"広島県, 千葉県, 静岡県, 宮城県, 福岡県, 滋賀県, 岡山県"
3,#4,"新潟県, 三重県, 群馬県, 山口県, 奈良県, 沖縄県, 愛媛県, 大分県, 富山県"
4,#5,"熊本県, 長崎県, 島根県, 茨城県, 鳥取県, 石川県, 香川県, 岐阜県, 栃木県, 佐賀県"
5,#6,"福井県, 山梨県, 宮崎県, 長野県, 岩手県, 山形県, 和歌山県, 鹿児島県, 北海道"
6,#7,"徳島県, 秋田県, 福島県, 高知県, 青森県"


In [350]:
print(df_tier.to_markdown())

|    | Tier   | 都道府県名                                                                     |
|---:|:-------|:-------------------------------------------------------------------------------|
|  0 | #1     | 東京都                                                                         |
|  1 | #2     | 神奈川県, 大阪府, 兵庫県, 京都府, 埼玉県, 愛知県                               |
|  2 | #3     | 広島県, 千葉県, 静岡県, 宮城県, 福岡県, 滋賀県, 岡山県                         |
|  3 | #4     | 新潟県, 三重県, 群馬県, 山口県, 奈良県, 沖縄県, 愛媛県, 大分県, 富山県         |
|  4 | #5     | 熊本県, 長崎県, 島根県, 茨城県, 鳥取県, 石川県, 香川県, 岐阜県, 栃木県, 佐賀県 |
|  5 | #6     | 福井県, 山梨県, 宮崎県, 長野県, 岩手県, 山形県, 和歌山県, 鹿児島県, 北海道     |
|  6 | #7     | 徳島県, 秋田県, 福島県, 高知県, 青森県                                         |
